In [1]:
# code for loading the format for the notebook
import os

# path : store the current path to convert back to it later
path = os.getcwd()
os.chdir('../../notebook_format')
from formats import load_style
load_style()

In [2]:
os.chdir(path)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = 8, 6 # change default figure size

# 1. magic for inline plot
# 2. magic to print version
# 3. magic so that the notebook will reload external python modules
%matplotlib inline
%load_ext watermark
%load_ext autoreload 
%autoreload 2

import joblib
from operator import itemgetter
from gensim.models import Word2Vec
from sklearn.datasets import fetch_20newsgroups

%watermark -a 'Ethen' -d -t -v -p numpy,pandas,gensim,sklearn,joblib

Using TensorFlow backend.


Ethen 2017-07-19 16:13:16 

CPython 3.5.2
IPython 5.4.1

numpy 1.13.1
pandas 0.20.2
gensim 2.2.0
sklearn 0.18.1
joblib 0.11


# Word2vec

`Word2Vec` is a unsupervised learning algorithm that uses a shallow neural network (with one hidden layer) to learn the vectorial representations of all the term words/phrases for a given corpus. The advantage that word2vec offers is tries to preserve the semantic meaning behind those terms. For example, a document may employ the words "dog" and "canine" to mean the same thing, but never use them together in a sentence. Ideally, the word2vec algorithm would be able to learn the context and place them together in similar vector semantic space.

gensim’s `Word2vec` expects a sequence of sentences as its input, where each sentence a list of words.

In [3]:
sentences = [['first', 'sentence'], ['second', 'sentence']]
model = Word2Vec(sentences, min_count = 2, size = 200, workers = 4)

The model also accepts several key parameters that affect both training speed and quality.

- `min_count`: For pruning the internal dictionary. Words that appear only once or twice in a billion-word corpus are probably uninteresting typos and garbage. In addition, there’s not enough data to make any meaningful training on those words, so it’s best to ignore them. A reasonable value for min_count is between 0-100, depending on the size of the dataset.
- `size`: Refers to the hidden layers size. Bigger size values require more training data, but can lead to better (more accurate) models. Reasonable values are in the tens to hundreds
- `workers`: Number of cores/threads used for training
- `window`: Only terms hat occur within a window-neighbourhood of a term, in a sentence, are associated with it during training. The usual value is 4. Unless your text contains big sentences, leave it at that.
- `sg` – This defines the algorithm. If equal to 1, the skip-gram technique is used. Else, the CBoW method is employed

The full list of parameters can be obtained [here](http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)

---

In the example above, keeping the input as a Python built-in list is convenient, but can use up a lot of RAM when the input is large.

Gensim only requires that the input must provide sentences sequentially, hence if our input files are scattered across several different places then instead of loading everything into an in-memory list, we can process the input file by file, line by line:

In [4]:
class Sentences:
    """
    iterate over files in a directory, and read in each line
    as a list of words. Used with gensim's Word2Vec

    Parameters
    ---------
    dirname: str
        directory that contains the file of text
    
    Examples
    --------
    # a memory-friendly iterator
    dirname = 'test'
    sentences = Sentences(dirname)
    model = Word2Vec(sentences)
    """
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for file in os.listdir(self.dirname):
            fname = os.path.join(self.dirname, file)
            with open(fname) as f:
                for line in f:
                    # we can also do other text preprocessing
                    # such as remove stop words, lower-case
                    # the strings, etc. here
                    yield line.split()

In [5]:
newsgroups_train = fetch_20newsgroups(subset = 'train')
newsgroups_train.data[0]

"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"

In [6]:
word2vec_model_path = 'mymodel'

workers = joblib.cpu_count()
documents = [doc.strip().split() for doc in newsgroups_train.data]

model = Word2Vec(documents, min_count = 2, size = 200, workers = workers)
model.save(word2vec_model_path)
model = Word2Vec.load(word2vec_model_path)

In [7]:
# acess the vocabulary attribute of 
# the word vector to build a list of the 
# terms, integer indices and term counts from model's vocabulary
ordered_vocab = [(term, info.index, info.count)
                 for term, info in model.wv.vocab.items()]

# sort by the term counts, so the most common terms appear first
ordered_vocab = sorted(ordered_vocab, key = itemgetter(2), reverse = True)

# unzip the terms, integer indices, and counts into separate lists
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)

# create a DataFrame with the learnt vectors as data,
# and the terms as row labels
word_vectors = pd.DataFrame(model.wv.syn0norm[term_indices, :],
                            index = ordered_terms)
print(word_vectors.shape)
word_vectors.head()

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# find the top 5 most similar term
model.wv.most_similar(positive = ['computer'], topn = 5)

# Reference

- [Blog: Word2vec API Tutorial](http://rare-technologies.com/word2vec-tutorial/)